# 2. OpenAI의 챗 API 기초


## 2.3. 입출력 길이 제한과 요금에 영향을 미치는 "토큰"


### 토큰


In [ ]:
!pip install tiktoken==0.7.0

In [ ]:
import tiktoken

text = "ChatGPT"

encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(text)
for token in tokens:
    print(encoding.decode([token]))

### 토크나이저와 tiktoken 소개


In [ ]:
import tiktoken

text = "LLM을 사용해 멋진 것을 만드는 것은 쉽지만, 프로덕션에서 사용할 수 있는 것을 만드는 것은 매우 어렵다."

encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(text)
print(len(tokens))

## 2.4. Chat Completions API 테스트 환경 준비


### OpenAI の API 키 준비


In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

## 2.5. Chat Completions API 실습


### OpenAI 라이브러리


#### 【주의】알려진 오류에 관해

openai 패키지가 의존하는 httpx의 업데이트로 인해, `openai==1.40.6`을 사용하는 부분에서 `TypeError: Client.__init__() got an unexpected keyword argument 'proxies'`라는 오류가 발생하게 되었습니다.

이 오류는 `!pip install httpx==0.27.2`와 같이, httpx의 특정 버전을 설치함으로써 회피할 수 있습니다.

또한, Google Colab에서 위 오류를 한 번 만난 후 `!pip install httpx==0.27.2`와 같이 패키지를 다시 설치한 경우, 다음 중 하나의 작업을 수행해야 합니다.

- Google Colab의 "런타임"에서 "세션 재시작하기"를 실행합니다
- "런타임 연결 해제 및 삭제"를 실행하여 패키지 설치부터 다시 시작합니다


In [ ]:
!pip install openai==1.40.6 httpx==0.27.2

### Chat Completions API 호출


In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "안녕하세요! 저는 존이라고 합니다!"},
    ],
)
print(response.to_json(indent=2))

### 대화 이력을 고려한 응답 얻기


In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "안녕하세요! 저는 존이라고 합니다!"},
        {"role": "assistant", "content": "안녕하세요, 존 님! 만나서 반갑습니다. 오늘은 어떤 이야기를 나눌까요?"},
        {"role": "user", "content": "제 이름을 아시나요?"},
    ],
)
print(response.to_json(indent=2))

### 스트리밍으로 응답 얻기


In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "안녕하세요! 저는 존이라고 합니다!"},
    ],
    stream=True,
)

for chunk in response:
    content = chunk.choices[0].delta.content
    if content is not None:
        print(content, end="", flush=True)

### JSON 모드


In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": '인물 목록을 다음 JSON 형식으로 출력해 주세요.\n{"people": ["aaa", "bbb"]}',
        },
        {
            "role": "user",
            "content": "옛날 옛적에 할아버지와 할머니가 살고 있었습니다",
        },
    ],
    response_format={"type": "json_object"},
)
print(response.choices[0].message.content)

### Vision(이미지 입력)


In [ ]:
from openai import OpenAI

client = OpenAI()

image_url = "https://raw.githubusercontent.com/ychoi-kr/langchain-book/main/cover.jpg"

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "이미지를 설명해 주세요."},
                {"type": "image_url", "image_url": {"url": image_url}},
            ],
        }
    ],
)

print(response.choices[0].message.content)

### (칼럼) Completions API


In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt="안녕하세요! 저는 존이라고 합니다!",
)
print(response.to_json(indent=2))

## 2.6. Function calling


### Function calling 샘플 코드


In [ ]:
import json


def get_current_weather(location, unit="fahrenheit"):
    if "tokyo" in location.lower():
        return json.dumps({"location": "Seoul", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps(
            {"location": "San Francisco", "temperature": "72", "unit": unit}
        )
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

In [ ]:
from openai import OpenAI

client = OpenAI()

messages = [
    {"role": "user", "content": "서울 날씨는 어떤가요?"},
]

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)
print(response.to_json(indent=2))

In [ ]:
response_message = response.choices[0].message
messages.append(response_message.to_dict())

In [ ]:
available_functions = {
    "get_current_weather": get_current_weather,
}

# 사용하고 싶은 함수는 여러 개일 수 있으므로 반복문 사용
for tool_call in response_message.tool_calls:
    # 함수를 실행
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(
        location=function_args.get("location"),
        unit=function_args.get("unit"),
    )
    print(function_response)

    # 함수 실행 결과를 대화 이력으로 messages에 추가
    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": function_response,
        }
    )

In [ ]:
print(json.dumps(messages, ensure_ascii=False, indent=2))

In [ ]:
second_response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
)
print(second_response.to_json(indent=2))